# Libraries

In [280]:
import pandas as pd
from getpass import getuser
from collections import defaultdict

# Load and inspect dataset

In [281]:
# Get the current user's name
user = getuser()

# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\fifa.csv'

# Read the dataset with a different encoding
df = pd.read_csv(data_path, encoding='ISO-8859-1')

# Extract relevant columns

In [282]:
# First, let's extract the year from the 'tournament_id' column and create a new column 'year'
df['year'] = df['tournament_id'].str.extract(r'WC-(\d{4})').astype(int)

# Now, filter the data for tournaments after 1986
filtered_df = df[df['year'] > 1986]

# Extract relevant columns for goal events and match results
goals_df = filtered_df[['tournament_name', 'group_name','match_name', 'match_id', 'player_team_name','match_date', 'minute_regulation', 
               'team_id', 'own_goal']]

# Remove all observations where group_name is "not applicable"
goals_df = goals_df[goals_df['group_name'] != 'not applicable']

# Convert 'match_date' to datetime format
goals_df['match_date'] = pd.to_datetime(goals_df['match_date'], format='%m/%d/%Y')

# Sort the dataset by 'match_date' in ascending order (oldest first) and 'minute_regulation'
goals_df = goals_df.sort_values(by=['match_date', 'minute_regulation'], ascending=[True, True])

# Display the first few rows to confirm the sorting
goals_df.head()


,tournament_name,group_name,match_name,match_id,player_team_name,match_date,minute_regulation,team_id,own_goal
1328,1990 FIFA World Cup,Group B,Argentina v Cameroon,M-1990-01,Cameroon,1990-06-08,67,T-11,0
1329,1990 FIFA World Cup,Group B,Soviet Union v Romania,M-1990-02,Romania,1990-06-09,41,T-59,0
1331,1990 FIFA World Cup,Group D,United Arab Emirates v Colombia,M-1990-03,Colombia,1990-06-09,50,T-15,0
1330,1990 FIFA World Cup,Group B,Soviet Union v Romania,M-1990-02,Romania,1990-06-09,55,T-59,0
1333,1990 FIFA World Cup,Group A,Italy v Austria,M-1990-04,Italy,1990-06-09,78,T-39,0


# Recreate Leauge Table after first two matchdays

### Filter out the matches from the last match day in the goals_df DataFrame.

In [283]:
# Step 1: Find the last match date for each tournament and group
last_dates = goals_df.groupby(['tournament_name', 'group_name'])['match_date'].max().reset_index()

# Step 2: Create the dataset with all games excluding the last match day (goals_before_last_day)
goals_before_last_day = goals_df.merge(last_dates, on=['tournament_name', 'group_name', 'match_date'], how='outer', indicator=True)
goals_before_last_day = goals_before_last_day[goals_before_last_day['_merge'] == 'left_only'].drop(columns=['_merge'])

# Step 3: Create the dataset with only the last match day games (goals_last_day)
goals_last_day = goals_df.merge(last_dates, on=['tournament_name', 'group_name', 'match_date'])

# Step 4: Split match_name into home and away teams and remove match_name column for both datasets

# For goals_before_last_day
goals_before_last_day[['home', 'away']] = goals_before_last_day['match_name'].str.split(' v ', expand=True)
goals_before_last_day = goals_before_last_day.drop(columns=['match_name'])

# For goals_last_day
goals_last_day[['home', 'away']] = goals_last_day['match_name'].str.split(' v ', expand=True)
goals_last_day = goals_last_day.drop(columns=['match_name'])


## create df  of aggregate data for each match

In [284]:
# Initialize an empty list to store the results
results = []

# Iterate over each match in the dataset
for match_id, group in goals_before_last_day.groupby(['tournament_name', 'group_name', 'match_id', 'home', 'away']):
    # Initialize goals_home and goals_away for each match
    goals_home = 0
    goals_away = 0
    
    # Loop through each row in the group
    for _, row in group.iterrows():
        # Check if the team is the home team and no own goal
        if row['home'] == row['player_team_name'] and row['own_goal'] == 0:
            goals_home += 1
        # If the team is the away team and no own goal, increment goals_away
        elif row['away'] == row['player_team_name'] and row['own_goal'] == 0:
            goals_away += 1

    # Append the results to the list, keeping home and away columns
    results.append({
        'tournament_name': match_id[0],
        'group_name': match_id[1],
        'match_id': match_id[2],
        'home': match_id[3],
        'away': match_id[4],
        'goals_home': goals_home,
        'goals_away': goals_away
    })

# Convert the list into a DataFrame
agg_goals_before_last_day = pd.DataFrame(results)

# Add a new column 'won' based on the comparison of goals_home and goals_away
agg_goals_before_last_day['won'] = agg_goals_before_last_day.apply(
    lambda row: 1 if row['goals_home'] > row['goals_away'] else (-1 if row['goals_home'] < row['goals_away'] else 0), 
    axis=1
)

# Display the updated DataFrame (use print if outside of Jupyter)
display(agg_goals_before_last_day)



,tournament_name,group_name,match_id,home,away,goals_home,goals_away,won
0,1990 FIFA World Cup,Group A,M-1990-04,Italy,Austria,1,0,1
1,1990 FIFA World Cup,Group A,M-1990-05,United States,Czechoslovakia,1,5,-1
2,1990 FIFA World Cup,Group A,M-1990-16,Italy,United States,1,0,1
3,1990 FIFA World Cup,Group A,M-1990-17,Austria,Czechoslovakia,0,1,-1
4,1990 FIFA World Cup,Group B,M-1990-01,Argentina,Cameroon,0,1,-1
...,...,...,...,...,...,...,...,...
244,2022 FIFA World Cup,Group G,M-2022-29,Cameroon,Serbia,3,3,0
245,2022 FIFA World Cup,Group G,M-2022-31,Brazil,Switzerland,1,0,1
246,2022 FIFA World Cup,Group H,M-2022-15,Portugal,Ghana,3,2,1
247,2022 FIFA World Cup,Group H,M-2022-30,South Korea,Ghana,2,3,-1


## aggregate data for home and away games

In [285]:
# Step 1: Home games aggregation
home_games = agg_goals_before_last_day.groupby(['tournament_name', 'group_name', 'home']).agg(
    goals_scored=('goals_home', 'sum'),
    goals_conceded=('goals_away', 'sum'),
    points_home=('won', lambda x: sum(3 if result == 1 else (1 if result == 0 else 0) for result in x))
).reset_index()

# Step 2: Away games aggregation
away_games = agg_goals_before_last_day.groupby(['tournament_name', 'group_name', 'away']).agg(
    goals_scored=('goals_away', 'sum'),
    goals_conceded=('goals_home', 'sum'),
    points_away=('won', lambda x: sum(3 if result == -1 else (1 if result == 0 else 0) for result in x))
).reset_index()


## aggregate data after first two matches

In [286]:
# Step 1: Merge home_games and away_games on tournament_name, group_name, and home with away
all_games_before_last = pd.merge(
    home_games,
    away_games,
    left_on=['tournament_name', 'group_name', 'home'],
    right_on=['tournament_name', 'group_name', 'away'],
    how='outer',
    suffixes=('_home', '_away')
)

# Step 2: For teams that only appear in home_games, assign home to team and copy relevant columns
all_games_before_last['team'] = all_games_before_last['home'].fillna(all_games_before_last['away'])

# Step 3: For goals_scored, goals_conceded, and points, handle missing values
all_games_before_last['goals_scored'] = all_games_before_last['goals_scored_home'].fillna(0) + all_games_before_last['goals_scored_away'].fillna(0)
all_games_before_last['goals_conceded'] = all_games_before_last['goals_conceded_home'].fillna(0) + all_games_before_last['goals_conceded_away'].fillna(0)
all_games_before_last['points'] = all_games_before_last['points_home'].fillna(0) + all_games_before_last['points_away'].fillna(0)

# Step 4: Add goals_difference column
all_games_before_last['goals_difference'] = all_games_before_last['goals_scored'] - all_games_before_last['goals_conceded']

# Step 5: Drop unnecessary columns used in the merge process
all_games_before_last = all_games_before_last[['tournament_name', 'group_name', 'team', 'goals_scored', 'goals_conceded', 'points', 'goals_difference']]

# Step 6: Sort by points, goals_difference, and goals_scored within each tournament and group
all_games_before_last = all_games_before_last.sort_values(
    by=['tournament_name', 'group_name', 'points', 'goals_difference', 'goals_scored'],
    ascending=[True, True, False, False, False]
).reset_index(drop=True)

# Display the final sorted dataframe
display(all_games_before_last)


,tournament_name,group_name,team,goals_scored,goals_conceded,points,goals_difference
0,1990 FIFA World Cup,Group A,Czechoslovakia,6.0,1.0,6.0,5.0
1,1990 FIFA World Cup,Group A,Italy,2.0,0.0,6.0,2.0
2,1990 FIFA World Cup,Group A,Austria,0.0,2.0,0.0,-2.0
3,1990 FIFA World Cup,Group A,United States,1.0,6.0,0.0,-5.0
4,1990 FIFA World Cup,Group B,Cameroon,3.0,1.0,6.0,2.0
...,...,...,...,...,...,...,...
266,2022 FIFA World Cup,Group G,Serbia,3.0,5.0,1.0,-2.0
267,2022 FIFA World Cup,Group H,Portugal,5.0,2.0,6.0,3.0
268,2022 FIFA World Cup,Group H,Ghana,5.0,5.0,3.0,0.0
269,2022 FIFA World Cup,Group H,South Korea,2.0,3.0,0.0,-1.0


In [287]:
goals_last_day

,tournament_name,group_name,match_id,player_team_name,match_date,minute_regulation,team_id,own_goal,home,away
0,1990 FIFA World Cup,Group B,M-1990-26,Soviet Union,1990-06-18,20,T-70,0,Cameroon,Soviet Union
1,1990 FIFA World Cup,Group B,M-1990-26,Soviet Union,1990-06-18,29,T-70,0,Cameroon,Soviet Union
2,1990 FIFA World Cup,Group B,M-1990-26,Soviet Union,1990-06-18,52,T-70,0,Cameroon,Soviet Union
3,1990 FIFA World Cup,Group B,M-1990-25,Argentina,1990-06-18,62,T-03,0,Argentina,Romania
4,1990 FIFA World Cup,Group B,M-1990-26,Soviet Union,1990-06-18,63,T-70,0,Cameroon,Soviet Union
...,...,...,...,...,...,...,...,...,...,...
344,2022 FIFA World Cup,Group G,M-2022-48,Serbia,2022-12-02,35,T-64,0,Serbia,Switzerland
345,2022 FIFA World Cup,Group G,M-2022-48,Switzerland,2022-12-02,44,T-73,0,Serbia,Switzerland
346,2022 FIFA World Cup,Group G,M-2022-48,Switzerland,2022-12-02,48,T-73,0,Serbia,Switzerland
347,2022 FIFA World Cup,Group H,M-2022-46,South Korea,2022-12-02,90,T-69,0,South Korea,Portugal


# Recreate League Table after last match day

In [288]:
# Step 1: Sort goals_last_day by minute_regulation in ascending order, grouped by tournament_name and group_name
goals_last_day_sorted = goals_last_day.sort_values(by=['tournament_name', 'group_name', 'match_id', 'minute_regulation'], ascending=True)

# Step 2: Create a new DataFrame to track overall and last match goals for each team
goals_tracking = pd.DataFrame(columns=['tournament_name', 'group_name', 'team', 
                                       'overall_goals_scored', 'overall_goals_conceded', 
                                       'last_match_goals_scored', 'last_match_goals_conceded', 
                                       'total_goal_difference', 'total_goals_scored', 'total_goals_conceded',
                                       'points_before_last_match'])

# Initialize the goals_tracking table by adding the teams from all_games_before_last
for _, row in all_games_before_last.iterrows():
    # Calculate points before the last match
    if row['goals_scored'] > row['goals_conceded']:
        points_before_last_match = 3  # Win
    elif row['goals_scored'] == row['goals_conceded']:
        points_before_last_match = 1  # Draw
    else:
        points_before_last_match = 0  # Loss

    # Add the team to the goals_tracking table with their overall goals from all_games_before_last
    new_row = pd.DataFrame({
        'tournament_name': [row['tournament_name']],
        'group_name': [row['group_name']],
        'team': [row['team']],
        'overall_goals_scored': [int(row['goals_scored'])],  # Initialize as overall goals scored
        'overall_goals_conceded': [int(row['goals_conceded'])],  # Initialize as overall goals conceded
        'last_match_goals_scored': [0],  # Initialize with 0
        'last_match_goals_conceded': [0],  # Initialize with 0
        'total_goal_difference': [int(row['goals_scored']) - int(row['goals_conceded'])],  # Calculate initial goal difference
        'total_goals_scored': [0],  # Initialize total goals scored column
        'total_goals_conceded': [0],  # Initialize total goals conceded column
        'points_before_last_match': [points_before_last_match]  # Initialize points before last match
    })
    goals_tracking = pd.concat([goals_tracking, new_row], ignore_index=True)

# Step 3: Iterate through the sorted and filtered last match goals and update the goals_tracking table
for _, goal in goals_last_day_sorted.iterrows():
    tournament_name = goal['tournament_name']
    group_name = goal['group_name']
    home_team = goal['home']
    away_team = goal['away']
    player_team = goal['player_team_name']
    own_goal = goal['own_goal']

    # Ensure that we are processing only goals from the same tournament and group
    if tournament_name in goals_tracking['tournament_name'].values and group_name in goals_tracking['group_name'].values:
        # Update the goals based on who scored the goal (own goal or normal goal)
        if own_goal == 0:  # Normal goal
            if player_team == home_team:
                # Home team scored, update home scored and away conceded
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_conceded'] += 1
            elif player_team == away_team:
                # Away team scored, update away scored and home conceded
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_conceded'] += 1
        elif own_goal == 1:  # Own goal
            if player_team == home_team:
                # Own goal by home team, away team scores
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_conceded'] += 1
            elif player_team == away_team:
                # Own goal by away team, home team scores
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_conceded'] += 1

# Step 4: Update total_goal_difference and add total goals scored and conceded
goals_tracking['total_goal_difference'] = (goals_tracking['overall_goals_scored'] + 
                                           goals_tracking['last_match_goals_scored']).astype(int) - \
                                          (goals_tracking['overall_goals_conceded'] + 
                                           goals_tracking['last_match_goals_conceded']).astype(int)

goals_tracking['total_goals_scored'] = (goals_tracking['overall_goals_scored'] + goals_tracking['last_match_goals_scored']).astype(int)
goals_tracking['total_goals_conceded'] = (goals_tracking['overall_goals_conceded'] + goals_tracking['last_match_goals_conceded']).astype(int)

# Final result: Display the updated goals tracking table
print("\nFinal goals_tracking table with points before the last match:")
print(goals_tracking)



Final goals_tracking table with points before the last match:
         tournament_name group_name            team overall_goals_scored  \
0    1990 FIFA World Cup    Group A  Czechoslovakia                    6   
1    1990 FIFA World Cup    Group A           Italy                    2   
2    1990 FIFA World Cup    Group A         Austria                    0   
3    1990 FIFA World Cup    Group A   United States                    1   
4    1990 FIFA World Cup    Group B        Cameroon                    3   
..                   ...        ...             ...                  ...   
266  2022 FIFA World Cup    Group G          Serbia                    3   
267  2022 FIFA World Cup    Group H        Portugal                    5   
268  2022 FIFA World Cup    Group H           Ghana                    5   
269  2022 FIFA World Cup    Group H     South Korea                    2   
270  2022 FIFA World Cup    Group H         Uruguay                    0   

    overall_goals_conced

In [289]:
# Specify the tournament_name and group_name you want to filter
tournament = "1990 FIFA World Cup"
group = "Group B"

# Filter the dataframe for the specific tournament and group
filtered_df = goals_tracking.loc[
    (all_games_before_last['tournament_name'] == tournament) & 
    (all_games_before_last['group_name'] == group)
]

# Print the filtered dataframe
display(filtered_df)


,tournament_name,group_name,team,overall_goals_scored,overall_goals_conceded,last_match_goals_scored,last_match_goals_conceded,total_goal_difference,total_goals_scored,total_goals_conceded,points_before_last_match
4,1990 FIFA World Cup,Group B,Cameroon,3,1,0,4,-2,3,5,3
5,1990 FIFA World Cup,Group B,Romania,3,2,1,1,1,4,3,3
6,1990 FIFA World Cup,Group B,Argentina,2,1,1,1,1,3,2,3
7,1990 FIFA World Cup,Group B,Soviet Union,0,4,4,0,0,4,4,0


In [290]:
# Step 1: Sort goals_last_day by minute_regulation in ascending order, grouped by tournament_name and group_name
goals_last_day_sorted = goals_last_day.sort_values(by=['tournament_name', 'group_name', 'match_id', 'minute_regulation'], ascending=True)

# Step 2: Create a new DataFrame to track overall and last match goals for each team
goals_tracking = pd.DataFrame(columns=['tournament_name', 'group_name', 'team', 
                                       'overall_goals_scored', 'overall_goals_conceded', 
                                       'last_match_goals_scored', 'last_match_goals_conceded', 
                                       'points_before_last_match', 'total_points', 
                                       'total_goal_difference', 'total_goals_scored', 'total_goals_conceded'])

# Initialize the goals_tracking table by adding the teams from all_games_before_last
for _, row in all_games_before_last.iterrows():
    # Add the team to the goals_tracking table with their overall goals from all_games_before_last
    new_row = pd.DataFrame({
        'tournament_name': [row['tournament_name']],
        'group_name': [row['group_name']],
        'team': [row['team']],
        'overall_goals_scored': [int(row['goals_scored'])],  # Initialize as overall goals scored
        'overall_goals_conceded': [int(row['goals_conceded'])],  # Initialize as overall goals conceded
        'last_match_goals_scored': [0],  # Initialize with 0
        'last_match_goals_conceded': [0],  # Initialize with 0
        'points_before_last_match': [int(row['points'])],  # Initialize points before last match
        'total_points': [int(row['points'])],  # Initialize total points as points before last match
        'total_goal_difference': [int(row['goals_scored']) - int(row['goals_conceded'])],  # Initialize goal difference
        'total_goals_scored': [int(row['goals_scored'])],  # Start with overall goals scored
        'total_goals_conceded': [int(row['goals_conceded'])]  # Start with overall goals conceded
    })
    goals_tracking = pd.concat([goals_tracking, new_row], ignore_index=True)

# Step 3: Iterate through the sorted and filtered last match goals and update the goals_tracking table
for _, goal in goals_last_day_sorted.iterrows():
    tournament_name = goal['tournament_name']
    group_name = goal['group_name']
    home_team = goal['home']
    away_team = goal['away']
    player_team = goal['player_team_name']
    own_goal = goal['own_goal']

    # Ensure that we are processing only goals from the same tournament and group
    if tournament_name in goals_tracking['tournament_name'].values and group_name in goals_tracking['group_name'].values:
        # Calculate and update the current ranking before processing the goal
        goals_tracking['total_goal_difference'] = goals_tracking['overall_goals_scored'] + goals_tracking['last_match_goals_scored'] - (goals_tracking['overall_goals_conceded'] + goals_tracking['last_match_goals_conceded'])

        previous_ranking = goals_tracking[['team', 'total_points', 'total_goal_difference', 'total_goals_scored']].sort_values(by=['total_points', 'total_goal_difference', 'total_goals_scored'], ascending=False)

        # Update the goals based on who scored the goal (own goal or normal goal)
        if own_goal == 0:  # Normal goal
            if player_team == home_team:
                # Home team scored, update home scored and away conceded
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_conceded'] += 1
            elif player_team == away_team:
                # Away team scored, update away scored and home conceded
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_conceded'] += 1
        elif own_goal == 1:  # Own goal
            if player_team == home_team:
                # Own goal by home team, away team scores
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_conceded'] += 1
            elif player_team == away_team:
                # Own goal by away team, home team scores
                goals_tracking.loc[(goals_tracking['team'] == home_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_scored'] += 1
                goals_tracking.loc[(goals_tracking['team'] == away_team) &
                                   (goals_tracking['tournament_name'] == tournament_name) &
                                   (goals_tracking['group_name'] == group_name), 'last_match_goals_conceded'] += 1

        # Step 4: Update total_goals_scored, total_goals_conceded, and total_goal_difference
        goals_tracking['total_goals_scored'] = (goals_tracking['overall_goals_scored'] + goals_tracking['last_match_goals_scored']).astype(int)
        goals_tracking['total_goals_conceded'] = (goals_tracking['overall_goals_conceded'] + goals_tracking['last_match_goals_conceded']).astype(int)
        goals_tracking['total_goal_difference'] = goals_tracking['total_goals_scored'] - goals_tracking['total_goals_conceded']

        # Step 5: Update total_points based on the last match result (reset to points_before_last_match first)
        for _, row in goals_tracking.iterrows():
            # Reset the total_points to points_before_last_match
            goals_tracking.loc[goals_tracking['team'] == row['team'], 'total_points'] = row['points_before_last_match']
            
            # Update total_points based on the last match result
            if row['last_match_goals_scored'] > row['last_match_goals_conceded']:
                # Add 3 points for a win
                goals_tracking.loc[goals_tracking['team'] == row['team'], 'total_points'] += 3  
            elif row['last_match_goals_scored'] == row['last_match_goals_conceded']:
                # Add 1 point for a draw
                goals_tracking.loc[goals_tracking['team'] == row['team'], 'total_points'] += 1  

        # Step 6: Compare the current ranking after updating and check for changes
        current_ranking = goals_tracking[['team', 'total_points', 'total_goal_difference', 'total_goals_scored']].sort_values(by=['total_points', 'total_goal_difference', 'total_goals_scored'], ascending=False)
        if not current_ranking.equals(previous_ranking):
            print(f"\nRanking change detected after goal by {player_team}:")
            print(current_ranking)

        # Reset points to points_before_last_match after comparison
        goals_tracking['total_points'] = goals_tracking['points_before_last_match']

# Final result: Display the updated goals tracking table with total points and goals
print("\nFinal goals_tracking table with total points and goals:")
print(goals_tracking)



Ranking change detected after goal by Austria:
                      team total_points  total_goal_difference  \
12            West Germany            7                      7   
168               Portugal            7                      7   
207                 Russia            7                      6   
56                  France            7                      6   
191                 France            7                      5   
..                     ...          ...                    ...   
78                 Jamaica            1                     -7   
122  Serbia and Montenegro            1                     -7   
170            North Korea            1                     -8   
234                 Panama            1                     -8   
39                  Greece            1                     -8   

     total_goals_scored  
12                    9  
168                   7  
207                   7  
56                    6  
191                   7  
.. 

KeyboardInterrupt: 

In [257]:
# Specify the tournament_name and group_name you want to filter
tournament = "1990 FIFA World Cup"
group = "Group B"

# Filter the dataframe for the specific tournament and group
filtered_df = goals_tracking.loc[
    (all_games_before_last['tournament_name'] == tournament) & 
    (all_games_before_last['group_name'] == group)
]

# Print the filtered dataframe
display(filtered_df)


,tournament_name,group_name,team,overall_goals_scored,overall_goals_conceded,last_match_goals_scored,last_match_goals_conceded,total_goal_difference,total_goals_scored,total_goals_conceded,points_before_last_match,total_points
4,1990 FIFA World Cup,Group B,Cameroon,3,1,0,4,-2,3,5,6,4
5,1990 FIFA World Cup,Group B,Romania,3,2,1,1,1,4,3,3,7
6,1990 FIFA World Cup,Group B,Argentina,2,1,1,1,1,3,2,3,6
7,1990 FIFA World Cup,Group B,Soviet Union,0,4,4,0,0,4,4,0,3
